<a href="https://colab.research.google.com/github/liuxiao916/Dive_into_DL_PyTorch/blob/main/4_1_%E6%A8%A1%E5%9E%8B%E6%9E%84%E5%BB%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 4.1 模型构建

本节参考 https://github.com/ShusenTang/Dive-into-DL-PyTorch/blob/master/docs/chapter04_DL_computation/4.1_model-construction.md

### 4.1.1 继承Modlue类来构造模型

In [4]:
import torch
import torch.nn as nn


In [2]:
class MLP(nn.Module):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784,256)
        self.act = nn.ReLU()
        self.output = nn.Linear(256,19)

    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

In [6]:
X = torch.rand(2,784)
net = MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=19, bias=True)
)


tensor([[-0.0551,  0.0100, -0.0518, -0.1623,  0.1930, -0.1366, -0.2694, -0.0795,
          0.1514, -0.0013,  0.0897, -0.0796, -0.0916,  0.0454, -0.1193,  0.0046,
          0.3908,  0.0832, -0.1977],
        [-0.1689,  0.0082, -0.0315, -0.1680,  0.1383,  0.0399, -0.3500, -0.0960,
          0.1791, -0.0713, -0.0400, -0.0597, -0.1473,  0.0493, -0.1545,  0.0805,
          0.3988, -0.0021, -0.0636]], grad_fn=<AddmmBackward>)

### 4.1.2 Module子类

1. Sequential

In [12]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self, *args):
        super(MySequential, self).__init__()
        if len(args) ==1  and isinstance(args[0], OrderedDict):
            for key,module in args[0].items():
                self.add_module(key, module)
        else:
            for idx, module in enumerate(args):
                self.add_module(str(idx), module)
    def forward(self, input):
        # self._modules返回一个 OrderedDict，保证会按照成员添加时的顺序遍历成员
        for module in self._modules.values():
            input = module(input)
        return input

In [13]:
net = MySequential(
        nn.Linear(784, 256),
        nn.ReLU(),
        nn.Linear(256, 10), 
        )
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0452, -0.0486,  0.3853, -0.3618,  0.0055, -0.0869, -0.0605, -0.0162,
         -0.0527, -0.2875],
        [-0.1462,  0.0566,  0.3440, -0.2721, -0.0286, -0.0554, -0.1379,  0.0618,
         -0.0851, -0.2249]], grad_fn=<AddmmBackward>)

2. ModuleList

In [14]:
net = nn.ModuleList([nn.Linear(784, 256), nn.ReLU()])
net.append(nn.Linear(256, 10)) 
print(net[-1])  
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [15]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10) for i in range(10)])

    def forward(self, x):
        # ModuleList can act as an iterable, or be indexed using ints
        for i, l in enumerate(self.linears):
            x = self.linears[i // 2](x) + l(x)
        return x

In [16]:
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList, self).__init__()
        self.linears = nn.ModuleList([nn.Linear(10, 10)])
    
class Module_List(nn.Module):
    def __init__(self):
        super(Module_List, self).__init__()
        self.linears = [nn.Linear(10, 10)]

net1 = Module_ModuleList()
net2 = Module_List()

print("net1:")
for p in net1.parameters():
    print(p.size())

print("net2:")
for p in net2.parameters():
    print(p)


net1:
torch.Size([10, 10])
torch.Size([10])
net2:


3. ModuleDict

In [18]:
net = nn.ModuleDict({
    'linear': nn.Linear(784, 256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256, 10) 
print(net['linear']) 
print(net.output)
print(net)


Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


### 4.1.3 构造复杂的网络

In [23]:
class FancyMLP(nn.Module):
    def __init__(self, *args):
        super(FancyMLP, self).__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, x):
        x = self.linear(x)
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)
        x = self.linear(x)
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()


In [24]:
X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)


FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-0.9318, grad_fn=<SumBackward0>)

In [25]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU()) 

    def forward(self, x):
        return self.net(x)

net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)


Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(6.2916, grad_fn=<SumBackward0>)